In [45]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [46]:
empresas = ['AMER3_3', 'MGLU3_3', 'PETR4_3']

In [47]:
# for empresa in empresas:
#     dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0')

#     dados['data_hora'] = dados['data_hora'].str.replace('.', '-')

#     dados['data_hora'] = dados['data_hora']+":00"

#     colunas = ['data_hora', 'url', 'titulo', 'texto']
#     dados_tratados = dados[colunas]

#     print(dados_tratados.head())
#     dados_tratados.to_csv(empresa + '.csv')


In [48]:
dados = pd.read_csv(empresas[0] + '.csv', index_col='Unnamed: 0')
dados['data_hora'] = pd.to_datetime(dados['data_hora'], dayfirst=True, format='%d-%m-%Y %H:%M:%S')


In [49]:
def getCsv(empresa):
    dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0')
    
    dados['data_hora'] = pd.to_datetime(dados['data_hora'], dayfirst=True, format='%d-%m-%Y %H:%M:%S')

    # dados['data_hora'] = pd.to_datetime(dados['data_hora'], dayfirst=True, format='%Y-%m-%d %H:%M:%S')
    
    return dados.sort_values('data_hora')

###################################################################################

In [50]:
def getDadosBolsa(empresa, dataMax, dataMin):

    dados = yf.Ticker(empresa + '.SA')
    dados = dados.history(start=(dataMin - timedelta(days=7)), end=(dataMax + timedelta(days=7)), interval='1d')
    dados.reset_index(inplace=True)
    dados['data_hora'] = dados['Date']
    dados.drop(columns=['Date'], inplace=True)
    dados.set_index('data_hora', inplace=True)
    return dados

In [51]:
def getDataMaisProxima(empresa, data):

    datas = {
                abs(data.timestamp() - date.timestamp()) : date for date in empresa.index}
    
    dataMaisProxima = datas[min(datas.keys())]

    return dataMaisProxima - timedelta(days=1), dataMaisProxima, dataMaisProxima + timedelta(days=1), dataMaisProxima + timedelta(days=2)

In [52]:
def getValores(dados, data):
    try:
        return dados.iloc[dados.index.get_loc(data)]

    except KeyError:
        return getValores(dados, data + timedelta(days=1))

def montaDictValores(valores):
    return {
        'Abertura': valores.Open,
        'Fechamento': valores.Close,
    }

def getValoresData(dados, data):
    print(getValores(dados, data))
    return montaDictValores(getValores(dados, data))

In [53]:
datetime.today().date()

datetime.date(2023, 10, 8)

In [54]:
def extraiValoresBolsa(empresa):
    df = getCsv(empresa)
    if '_' in empresa:
        dados_bolsa = getDadosBolsa(empresa[:-2], df['data_hora'].max(), df['data_hora'].min())
    else:
        dados_bolsa = getDadosBolsa(empresa, df['data_hora'].max(), df['data_hora'].min())

    aberturaDiaAnterior = []
    aberturaDiaAtual = []
    aberturaDiaSeguinte = []
    abertura2Dias = []

    fechamentoDiaAnterior = []
    fechamentoDiaAtual = []
    fechamentoDiaSeguinte = []
    fechamento2Dias = []

    print(df.shape)
    i = 0
    for data in df['data_hora']:
        
        diaAnterior, dataAtual, dataSeguinte, dia2 = getDataMaisProxima(dados_bolsa, data)

        valores = getValoresData(dados_bolsa, diaAnterior)
        aberturaDiaAnterior.append(valores['Abertura'])
        fechamentoDiaAnterior.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dataAtual)
        aberturaDiaAtual.append(valores['Abertura'])
        fechamentoDiaAtual.append(valores['Fechamento'])

        # print(dataSeguinte)
        valores = getValoresData(dados_bolsa, dataSeguinte)
        # print(valores)
        aberturaDiaSeguinte.append(valores['Abertura'])
        fechamentoDiaSeguinte.append(valores['Fechamento'])
        # print(i)
        # i += 1
        
        valores = getValoresData(dados_bolsa, dia2)
        abertura2Dias.append(valores['Abertura'])
        fechamento2Dias.append(valores['Fechamento'])

        

    print('cheguei aqui') 
    
    df['aberturaDiaAnterior'] = aberturaDiaAnterior
    df['fechamentoDiaAnterior'] = fechamentoDiaAnterior

    df['aberturaDiaAtual'] = aberturaDiaAtual
    df['fechamentoDiaAtual'] = fechamentoDiaAtual

    df['aberturaDiaSeguinte'] = aberturaDiaSeguinte
    df['fechamentoDiaSeguinte'] = fechamentoDiaSeguinte

    df['abertura2Dias'] = abertura2Dias
    df['fechamento2Dias'] = fechamento2Dias

    return df

In [55]:
for empresa in empresas:
    print(empresa)
    df = extraiValoresBolsa(empresa)
    df.to_csv(empresa + '.csv')

AMER3_3


(25, 4)
Open                   1.14
High                   1.17
Low                    1.12
Close                  1.16
Volume          14133600.00
Dividends              0.00
Stock Splits           0.00
Name: 2023-06-29 00:00:00-03:00, dtype: float64
Open                   1.18
High                   1.21
Low                    1.15
Close                  1.17
Volume          19908300.00
Dividends              0.00
Stock Splits           0.00
Name: 2023-06-30 00:00:00-03:00, dtype: float64
Open                   1.18
High                   1.19
Low                    1.15
Close                  1.19
Volume          11072200.00
Dividends              0.00
Stock Splits           0.00
Name: 2023-07-03 00:00:00-03:00, dtype: float64
Open                   1.18
High                   1.19
Low                    1.15
Close                  1.19
Volume          11072200.00
Dividends              0.00
Stock Splits           0.00
Name: 2023-07-03 00:00:00-03:00, dtype: float64
Open            